You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

In [4]:
# Instalação das dependências
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

  Using cached crewai-0.28.8-py3-none-any.whl.metadata (13 kB)
  Using cached crewai_tools-0.1.6-py3-none-any.whl.metadata (4.6 kB)
  Using cached langchain_community-0.0.29-py3-none-any.whl.metadata (8.3 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached embedchain-0.1.127-py3-none-any.whl.metadata (9.3 kB)
  Using cached instructor-0.5.2-py3-none-any.whl.metadata (10 kB)
  Using cached langchain-0.1.20-py3-none-any.whl.metadata (13 kB)
  Using cached opentelemetry_api-1.30.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached opentelemetry_exporter_otlp_proto_http-1.30.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached opentelemetry_sdk-1.30.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached python_dotenv-1.0.0-py3-none-any.whl.metadata (21 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 698.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Ge

In [5]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [7]:
# Importando classes necessárias para o kick-off
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [8]:
# Adicionando a API da OpenAI às variáveis de ambiente
import os
from google.colab import userdata

openai_api_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [9]:
# Criação do agente planejador
planner = Agent(
    role="Content Planner", # Definição do papel
    goal="Plan engaging and factually accurate content on {topic}", # Objetivo do agente
    backstory="You're working on planning a blog article " # "História prévia" do agente, basicamente o seu prompt do sistema principal
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)
# Este agente será responsável por executar, de fato, a pesquisa sobre o tópico, e planejar o artigo

### Agent: Writer

In [10]:
# Criação do agente escritor
# Este agente irá fazer a escrita dos tópicos para o artigo
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provided by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [11]:
# Criação do agente editor
# Este agente é o caminho final da equipe, ele edita o post e faz uma revisão/reescrita das informações passadas a ele pelo escritor
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [12]:
# Tarefa de planejamento -> agente planejador
# Esta tarefa é a inicial, passada para o planejador, para guiá-lo a passar o documento de planejamento para o escritor
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [13]:
# Tarefa de escrita -> agente escritor
# Esta tarefa é passada para estruturar a resposta do agente escritor, e guiá-lo na escrita dos tópicos
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [14]:
# Tarefa de edição -> agente editor
# Esta tarefa guia o editor a a corrigir e alinhar o artigo escrito pelo escritor
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution.

In [15]:
# Criação da equipe, com os respectivos agentes e tarefas
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [16]:
# Passando o tópico para a equipe e dando o início aos trabalhos
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: The Future of Artificial Intelligence: Latest Trends, Key Players, and Noteworthy News

Introduction:
- Brief overview of Artificial Intelligence (AI) and its impact on various industries
- Importance of staying updated on the latest trends and key players in the AI space

Key Points:
1. Latest Trends in Artificial Intelligence:
- Machine learning advancements and applications
- Natural language processing and chatbots
- AI ethics and regulation developments
- AI in healthcare, finance, and other sect

- Display the results of your execution as markdown in the notebook.

In [17]:
# Resultado final do artigo
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Latest Trends, Key Players, and Noteworthy News

## Introduction

Artificial Intelligence (AI) has become a transformative force across various industries, revolutionizing the way we work, communicate, and interact with technology. From machine learning advancements to natural language processing, AI is reshaping our world at an unprecedented pace. Staying updated on the latest trends and key players in the AI space is crucial for businesses, professionals, and enthusiasts alike to navigate the rapidly evolving landscape of AI technology.

## Latest Trends in Artificial Intelligence

One of the most significant trends in AI is the continuous advancement of machine learning algorithms and their applications across different sectors. From self-driving cars to personalized recommendations, machine learning is driving innovation and efficiency in various industries. Additionally, natural language processing and chatbots have become essential tools for improving customer service and streamlining communication processes.

Ethical considerations and regulatory developments in AI have also gained prominence in recent years. As AI technologies become more integrated into our daily lives, ensuring transparency, accountability, and fairness in their use is paramount. Furthermore, AI's impact on healthcare, finance, and other sectors is undeniable, with the potential to revolutionize patient care, financial services, and more.

## Key Players in the AI Industry

Leading tech giants such as Google, Amazon, and Microsoft are at the forefront of AI innovation, investing heavily in research and development to push the boundaries of what is possible with AI technology. Alongside these industry giants, emerging startups are making significant contributions to the AI space, disrupting traditional business models and driving forward new AI applications.

Key individuals, including researchers, entrepreneurs, and policymakers, are shaping the future of AI technology through their work and advocacy for responsible AI development. Their contributions are instrumental in driving progress and ensuring that AI technology is developed and deployed in a way that benefits society as a whole.

## Noteworthy News in Artificial Intelligence

Recent breakthroughs in AI research, such as advancements in deep learning and reinforcement learning, are pushing the boundaries of what AI can achieve. Industry partnerships and collaborations are also on the rise, with companies and organizations working together to harness the power of AI for social good and economic growth. The impact of AI on society and the workforce is a topic of ongoing debate, with discussions around job displacement, bias in AI algorithms, and the ethical implications of AI technology.

## Conclusion and Call to Action

In conclusion, Artificial Intelligence is set to shape the future of technology and society in profound ways. By staying informed about the latest trends, key players, and noteworthy news in the AI space, individuals and organizations can position themselves to thrive in the AI-driven world. Embracing the opportunities and challenges presented by AI, while also upholding ethical standards and regulatory frameworks, will be essential for realizing the full potential of AI technology.

As we look to the future of Artificial Intelligence, it is crucial for all stakeholders to work together to ensure that AI is developed and deployed responsibly, ethically, and inclusively. By harnessing the power of AI for the greater good, we can create a future where AI technology benefits everyone and drives positive change in society.

---
This blog post has been crafted following the comprehensive content plan provided, incorporating SEO keywords naturally and ensuring the content is structured with an engaging introduction, insightful body, and a summarizing conclusion. The post has been proofread for grammatical errors and aligned with the brand's voice, providing valuable insights and information to the target audience interested in the future of AI.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [18]:
# Tópico de teste pessoal -> criptomoedas
topic = "Crypto currencies"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Crypto currencies.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan: 
Title: The Ultimate Guide to Crypto Currencies

Outline:
I. Introduction
- Definition of Crypto currencies
- Brief history of Crypto currencies
- Importance of Crypto currencies in the current financial landscape

II. Latest Trends in Crypto currencies
- Rise of NFTs (Non-Fungible Tokens)
- Growth of decentralized finance (DeFi)
- Increased institutional adoption of Crypto currencies

III. Key Players in the Crypto currency Market
- Bitcoin (BTC)
- Ethereum (ETH)
- Ripple (XRP)
- Litecoin (LTC)
- Car

In [19]:
Markdown(result)

# The Ultimate Guide to Crypto Currencies

## Introduction

Cryptocurrencies, also known as digital or virtual currencies, have been revolutionizing the financial landscape in recent years. These digital assets leverage cryptography for secure transactions, operating independently of traditional banking systems and government control. The inception of Bitcoin in 2009 by the mysterious Satoshi Nakamoto marked the beginning of a new era in finance, leading to the widespread adoption and growth of cryptocurrencies.

The significance of cryptocurrencies lies in their ability to enhance financial inclusivity, reduce transaction fees, and facilitate faster cross-border transactions. As we explore the world of cryptocurrencies, it becomes crucial to grasp the latest trends that are shaping this dynamic industry.

## Latest Trends in Cryptocurrencies

One of the prominent trends in the cryptocurrency space is the emergence of Non-Fungible Tokens (NFTs), unique digital assets that represent ownership of specific items or content. NFTs have gained traction in industries like art and gaming, offering creators new avenues to monetize their work and collectors to own exclusive digital assets.

Another trend worth monitoring is the expansion of decentralized finance (DeFi) platforms, which aim to reimagine traditional financial systems through blockchain technology. These platforms provide services like lending, borrowing, and trading without intermediaries, granting users more control over their financial assets and decisions.

Furthermore, institutional adoption of cryptocurrencies has been increasing, with major companies and financial institutions investing in digital assets. This growing interest from institutions lends credibility and validation to the cryptocurrency market, fostering greater acceptance among mainstream investors.

For the latest updates and insights on the cryptocurrency market, it is advisable to follow reputable sources and explore the potential benefits of cryptocurrency investments.

## Key Players in the Cryptocurrency Market

### Bitcoin (BTC)
Bitcoin, the pioneering cryptocurrency, is often referred to as digital gold. It functions as a peer-to-peer electronic cash system, enabling secure and transparent transactions without intermediaries.

### Ethereum (ETH)
Ethereum is a decentralized platform that empowers developers to create and deploy smart contracts and decentralized applications (DApps). It has played a pivotal role in the growth of the DeFi ecosystem and offers innovative features like programmable money.

### Ripple (XRP)
Ripple serves as a digital payment protocol designed for swift and cost-effective cross-border transactions. It strives to streamline global money transfers for financial institutions using blockchain technology.

### Litecoin (LTC)
Litecoin, a peer-to-peer cryptocurrency, serves as a lightweight alternative to Bitcoin. It boasts faster transaction speeds and lower fees, making it a preferred choice for everyday transactions.

### Cardano (ADA)
Cardano is a blockchain platform committed to delivering a secure and scalable infrastructure for DApp and smart contract development. It leverages a proof-of-stake consensus mechanism to achieve sustainability and efficiency.

As these key players continue to innovate and shape the cryptocurrency market, they drive advancements in blockchain technology, paving the way for a transformative future.

## Noteworthy News in Cryptocurrencies

### Regulatory Updates
Regulatory changes play a pivotal role in influencing the cryptocurrency market. Recent updates from governments and regulatory bodies focus on enhancing transparency, combating fraud, and safeguarding investors. It is imperative for market participants to stay abreast of these developments to navigate the evolving regulatory landscape successfully.

### Major Developments in Blockchain Technology
Blockchain technology, the foundation of cryptocurrencies, is continuously evolving and revolutionizing diverse industries. Innovations like layer 2 solutions, interoperability protocols, and decentralized storage are propelling blockchain adoption and scalability to new heights.

### Market Analysis and Predictions
Industry experts and analysts offer valuable insights into market trends, price fluctuations, and future forecasts for cryptocurrencies. By staying informed about market analysis and predictions, investors can make informed decisions and capitalize on opportunities within the cryptocurrency market.

In conclusion, the cryptocurrency industry is dynamic and fast-paced, with evolving trends, key players, and technological advancements shaping its trajectory. By staying informed and educated about the latest news and developments, investors can navigate the cryptocurrency market confidently and make informed decisions regarding their financial future. Stay tuned for more updates and insights into the captivating world of cryptocurrencies.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```